# Exercise 2 - RNA editing detection using sequencing data
##  Giant fiber lobe (GFL) notebook
### submit: David Gorelik

In [25]:
import PySAM as pysam #import pysam as the other version didn't 
from scipy.stats import binom
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
print("Finished imports")

Finished imports


In [2]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 6600-1BAB

 Directory of C:\Users\zbida\OneDrive - Bar-Ilan University - Students\Documents\Documents_general\Cloud\Bar-Ilan\courses\Neurogenomics\Untitled Folder

05/03/2022  07:10 PM    <DIR>          .
05/03/2022  07:10 PM    <DIR>          ..
05/03/2022  12:02 PM    <DIR>          .ipynb_checkpoints
05/03/2022  12:04 PM            61,178 Class_editing_analysis.ipynb
05/03/2022  12:05 PM            60,749 Class_editing_analysis_gills.ipynb
05/03/2022  07:10 PM             6,377 exe2_squid.ipynb
05/03/2022  01:59 PM    <DIR>          GFL
05/03/2022  11:56 PM    <DIR>          Gills
05/03/2022  12:03 PM        23,089,234 pealeii.txt
               4 File(s)     23,217,538 bytes
               5 Dir(s)   4,803,829,760 bytes free


<div class="alert alert-block alert-info">
<b>q1:</b> Read the transcriptome file, i.e. all the genes of the squid, and collect the open reading frame and the RNA sequence of each gene.
</div>

First we daclare a helper class and then we collect the transcriptome data

In [5]:
#helper class fpr the transcriptome file, contains multiple variables
class Transcriptome_gene:
    def __init__(self, name, orfStart, orfEnd, strand, GO, seq, seq_ORF = None):
        self.name = name
        self.orfStart = orfStart
        self.orfEnd = orfEnd
        self.strand = strand
        self.GO = GO
        self.seq = seq
        self.seq_ORF = seq_ORF
    
path_squid_transcripts = "pealeii.txt" # the file located locally in the same dir as the notebook.

In [6]:
#Now to read the file using biopython, and then save each gene in dict 
dict_genes = {} #daclare dict
with open(path_squid_transcripts) as handle:
    for record in SeqIO.parse(handle, "fasta"):
        # We will devide the description into it's arguments
        list_items = ((record.description).split("\t"))
        name = list_items[0]
        orfStart = int(list_items[2])
        orfEnd = int(list_items[4])
        strand = list_items[6]
        GO = list_items[7]
        seq = record.seq #biopython seq
        seq_ORF = seq[orfStart-1:orfEnd -1]
        
        # use the defined class and save each item.
        gene_transcript = Transcriptome_gene(name = name, orfStart = orfStart, orfEnd = orfEnd, 
                                                strand = strand,GO = GO, seq = seq, seq_ORF = seq_ORF)
        # Now we will save the final item in a dict with key 
        dict_genes[name] = gene_transcript

Now we will give an example to the usage of the dict

In [18]:
for key_gene_name in dict_genes:
    print ("Our gene name are:", key_gene_name, "We access the data as:")
    print(dict_genes[key_gene_name].seq_ORF)
    break

Our gene name are: comp132287_c0_seq3 We access the data as:
AAAAATACAATAAAAAGAAAAAAATACAAAAATAACCCTAAACGGTTTCAGTTATTCAACACGATGGTCGAGGCACTATCAGCAGTCACAATTAACGGCACCCTGGCCCGTATGGGGCTCAACTTTACAGATGCTGACAAGGAGGGCGATGAGGCCGACCACGAAGCACATAGTCTGGACAGAAACATGGAACGCATTGTTGTGCCGATCTTATTTGGCGTTGCGTTCATTGTCGGTATTGTCGGGAATGGCACCCTCATCTATACAGTCTTGCGTAACCGAAAAATGAGAGTGGTCCCTAACATCTATATTGTAAGTCTATCTTGCGGCGACTTCTTACTCATACTCATTGCTGTACCCTTCAACGCCCTTATTTATATCTTACCGGAATGGCCGTTTGGAGAAATCATGTGTAAGGTCAACGAATACCTCCAAACTGTATCCCTAGGGGTTTCCGTATTTATGCTCACAGCTCTGAGCGCCGATCGGCATATTGCTATTGTGGACCCTATAGCTAAGCACAAATCAAGGCCCATCGTCCGCGCAGTCACCACAGCCGGCTGCTTGTGGCTAGTGGCTCTTTTGTTAGGCATACCGGATTTAAGTTCGTCGACCGTCATCCAGTTCTCACCCAACATCACCCTTGGATCTTACAAGGTTTGCATCTTATACCCGACCTCGGATTTCGGAGGGTACTTACCGCCGTGGTATTCGCAACTGATGGTCATGCTCAAGTTTTTCATCTTCTTCCTCGTGCCGCTGTTCATCATCGGCGCCTTCTACATCCTGATGGCGCGGATCCTCATCCTGAGCGCCAAACAGATCCCGGGCGATTCGAACGGGCGAGCGGCCTACCAGAAACAGATCGAGGCGCGCCTCAAGGTCGCCAAGGCGGTCCTGTCGTTCGTCGTCCTCTTCGTAATCTGCTGGCTACCCCGT

<div class="alert alert-block alert-info">
<b>q2:</b> Run over all the genes that are present in the transcriptome file. Print to file the name of the gene that is being examined
    
<b>q3:</b> For each gene, run over all the gene locations that are inside the open reading frame using pysam pileup. We are only looking for modifications inside the open reading frame, i.e. modifications in the untranslated regions should not be detected.
    
<b>q4:</b> For each location, you should examine all the reads that are uniquely mapped to that location (more accurately: reads that are uniquely mapped to that gene and the alignment contain that specific location). In addition, demand a quality score of 30 or above for the aligned read in that location.
    
<b> q5: </b> For each location, you should calculate how many counts (i.e. aligned reads that passed the criteria above) are there for each base: A, C, T, and G. Sort the counts for each base from the highest to the lowest. The highest will be treated as the ‘unmodified’ base, and the second highest will be treated as the ‘modified’ base. Say that A is the ‘unmodified’ base and G is the ‘modified’ base, the ‘modification’ is therefore AG (A-to-G).


</div>

We first daclare our variables as requseted:

In [19]:
### Code based and copied from the given notebook in the assigment

### start of input
minimum_qaulity_score=30 # quality score 30 corresponds to 1/1000 error rate, or 1e-3 sequencing error 
P=1e-3 #  propability of sequencing error given qaulity score of 30
max_mapping_score=42 # the maximal mapping quality means unique alignment, with bowtie2 this number is 42
p_cutoff=0.05 # the cutoff for significance, note that bonferroni correction is used below

bam_file_to_read="GFL/aligned_RNAreads_sorted.bam"
output_file="GFL/editing_sites_spectrin.txt"
length_of_read=100
total_orf_size=16607506

counts_per_letter={} # initilize dictionary for counts per letter
number_of_modifications={} # initilize dictionary for type of modification
type_of_modifications=['AC','AG','AT','CA','CG','CT','GA','GC','GT','TA','TC','TG']
for x in type_of_modifications:
    number_of_modifications[x] = 0
position_of_mismatch_inside_the_read=np.zeros(length_of_read) # initilize numpy array for position of mismatch in the read
position_of_match_inside_the_read=np.zeros(length_of_read) # initilize numpy array for position of match in the read

In [26]:
samfile = pysam.AlignmentFile(bam_file_to_read,"rb") # read the indexed and sorted bam file using pysam 

AttributeError: module 'PySAM' has no attribute 'AlignmentFile'

In [20]:
### use pysam to examine all the reads aligned for each position of each transcipt and detect editing sites
samfile = pysam.AlignmentFile(bam_file_to_read,"rb") # read the indexed and sorted bam file using pysam 
file2 = open(output_file,"w") # file in which the detected editing sited will be listed 
transcript_name='comp141540_c0_seq3' # in case only one transcript is run ('comp141540_c0_seq3' is spectrin)

# run over all transcripts
file2.write("\nname of transcript = %s\n" % transcript_name) # list the current transcript
# run over all positions of the transcript and fetch the reads aligned to it 
# only fetch reads that have the maximal mapping score (i.e. unique alignment)
for pileupcolumn in samfile.pileup(transcript_name,min_mapping_quality=max_mapping_score): 
    position=pileupcolumn.pos
    # initilize the dictionary counts_per_letter
    type_of_letters=['A','C','G','T']
    for x in type_of_letters:
        counts_per_letter[x]=0
    # run over all reads that align against the transcript in the given position
    for pileupread in pileupcolumn.pileups:
        # query position is None if is_del or is_refskip is set
        if not pileupread.is_del and not pileupread.is_refskip:
            # demand minimum quality score for the aligned base
            if pileupread.alignment.query_qualities[pileupread.query_position]>=minimum_qaulity_score:
                # detect the base
                letter=pileupread.alignment.query_sequence[pileupread.query_position]
                # count how many times each base was detected
                counts_per_letter[letter]=counts_per_letter[letter]+1 
    # sort the counts per base to detect the base most frequent and the second base i.e. the modification
    sorted_dict = dict( sorted(counts_per_letter.items(),
                               key=lambda item: item[1],
                               reverse=True)) # sort descending
    i=0 # a running number to detect the first and second most frequent bases
    N=0 # total number of reads aligned to the first and second most frequent bases 
    for key, value in sorted_dict.items():
        i=i+1 
        if i==1:
            N=N+value
            first_letter=key # the most frequent base
        elif i==2:
            X=value # the number of times the modified base appeared
            N=N+value
            second_letter=key # the second most frequent base
            modification=first_letter+second_letter # the modification, for example AG (i.e A changed to G)
        else:
            break
    # given a rate of sequencing error P, total number of counts N, and modified counts X \
    #  the probability of event being a sequencing error is 1 - binomial_CDF(X-1,N,P) \
    #  this is the same as binomial_CDF(N-X,N,1-P) (but the latter is more accurate due to rounding errors) 
    prob = binom.cdf(N-X,N,1-P)  
    # bonferroni correction p_cutoff divided by the overall number of tests
    if prob<(p_cutoff/total_orf_size):
        # print the total number of reads aligned for the specfic location inside the transcript
        file2.write("\ncoverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))  
        # sort the number of bases and print them to file
        for key, value in sorted_dict.items():
            file2.write('\ncounts of base %s = %s' % (key,str(value)))
        file2.write('\nX is %s and N is %s' % (str(X),str(N))) # print X and N, as defined above
        part1='\nP-value for sequencing error is\t' 
        part2="{:.2e}".format(prob)
        part3='\n'
        file2.write(part1+part2+part3) # print the P-value
        # update the number of modifications
        number_of_modifications[modification]=number_of_modifications[modification]+1
        # check the location of the match and mismatch in the read 
        #  to estimate if the modifications came from the reads ends 
        # run over all reads that align against the transcript in the given position 
        for pileupread in pileupcolumn.pileups:
            # query position is None if is_del or is_refskip is set
            if not pileupread.is_del and not pileupread.is_refskip:
                # demand minimum quality score for the aligned base
                if pileupread.alignment.query_qualities[pileupread.query_position]>=minimum_qaulity_score:
                    # detect the base
                    letter=pileupread.alignment.query_sequence[pileupread.query_position]
                    if (letter==first_letter): # these are reads that show the match
                        position_of_match_inside_the_read[pileupread.query_position]=\
                        position_of_match_inside_the_read[pileupread.query_position]+1
                    elif (letter==second_letter): # these are reads that show the mismatch
                        position_of_mismatch_inside_the_read[pileupread.query_position]=\
                        position_of_mismatch_inside_the_read[pileupread.query_position]+1
file2.close()

NameError: name 'pysam' is not defined